# Mink usage examples

This notebook demonstrates some examples of using mink and how it interacts with sklearn.

## Imports

In [1]:
import pickle

In [2]:
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.datasets import make_classification
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [3]:
from mink import NeuralNetClassifier
from mink import NeuralNetRegressor
from mink.layers import DenseLayer
from mink.layers import InputLayer
from mink.updates import SGD

## MNIST classification task

### Data

In [4]:
X, y = make_classification(n_samples=5000)

### Define neural net

Note: No need to specify the shape of the training data, number of classes, or to set softmax nonlinearity. The `NeuralNetClassifier` class takes care of all of that, as is expected from an sklearn estimator.

In [5]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [6]:
net = NeuralNetClassifier(l2, verbose=1)

If we want to change certain parameters after initialization, just use the `set_params` method and the double-underscore notation known from sklearn.

In [7]:
net.set_params(update__learning_rate=0.5)

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fcf6ac58e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fcf6ac9f9b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

### Train

In [8]:
net.fit(X, y, epochs=0)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |



NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fcf6ac58e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fcf6ac9f9b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [9]:
y_proba = net.predict_proba(X)

In [10]:
(np.argmax(y_proba, 1) == y).mean()

0.60199999999999998

In [11]:
net.fit(X, y, epochs=20)

# Neural Network with 4602 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |     20 |
|   1 | dense  |    200 |
|   2 | dense  |      2 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |      0.18051 | 0.04445 |
|       2 |      0.13042 | 0.03675 |
|       3 |      0.12566 | 0.03759 |
|       4 |      0.12196 | 0.03696 |
|       5 |      0.11886 | 0.03512 |
|       6 |      0.11603 | 0.03827 |
|       7 |      0.11343 | 0.03691 |
|       8 |      0.11101 | 0.03551 |
|       9 |      0.10867 | 0.03741 |
|      10 |      0.10650 | 0.03952 |
|      11 |      0.10446 | 0.03928 |
|      12 |      0.10253 | 0.03901 |
|      13 |      0.10071 | 0.03611 |
|      14 |      0.09902 | 0.03718 |
|      15 |      0.09738 | 0.03837 |
|      16 |      0.09579 | 0.03829 |
|      17 |      0.09426 | 0.03807 |
|      18 |      0.09275 | 0.03787 |
|      19 |      0.09112 | 0.03831 |
|      20 |      0.08

NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Softmax(), num_units=2),
          max_epochs=10, objective=CrossEntropy(eps=1e-12),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fcf6ac58e48>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fcf6ac9f9b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.5), verbose=1)

In [12]:
y_proba = net.predict_proba(X)

In [13]:
(np.argmax(y_proba, 1) == y.flatten()).mean()

0.97799999999999998

### Grid search

The neural net estimators can be used in conjunction with other sklearn features, such as `GridSearchCV`.

In [14]:
l0 = InputLayer()
l1 = DenseLayer(l0, name='hidden')
l2 = DenseLayer(l1)

In [15]:
net = NeuralNetClassifier(l2, update=SGD())

Parameters are set using the known double-underscore notation, e.g.

`'update__learning_rate': [0.1, 0.5]`.

Note: Instead of having to write

`'layer__incoming__num_units': [50, 100]`

we can just write

`'hidden__num_units': [50, 100]`

because we have given the hidden layer a name, "hidden". This may safe a lot of writing and confusion.

In [16]:
params = {
    'update__learning_rate': [0.1, 0.5],
    'max_epochs': [5, 10],
    'hidden__num_units': [50, 100],
}

In [17]:
cv = GridSearchCV(net, params, scoring='accuracy', refit=False, verbose=3)

In [18]:
cv.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.968206 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.965207 -   0.3s
[CV] max_epochs=5, update__learning_rate=0.1, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.1, hidden__num_units=50, score=0.956783 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.965807 -   0.4s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hidden__num_units=50, score=0.965207 -   0.5s
[CV] max_epochs=5, update__learning_rate=0.5, hidden__num_units=50 ...
[CV]  max_epochs=5, update__learning_rate=0.5, hi

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   18.1s finished


GridSearchCV(cv=None, error_score='raise',
       estimator=NeuralNetClassifier(batch_iterator_test=128, batch_iterator_train=128,
          encoder=LabelBinarizer(neg_label=0, pos_label=1, sparse_output=False),
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=Fa...ject at 0x7fcf6ac9f9b0>,),
          session_kwargs=None, update=SGD(learning_rate=0.01), verbose=0),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_epochs': [5, 10], 'update__learning_rate': [0.1, 0.5], 'hidden__num_units': [50, 100]},
       pre_dispatch='2*n_jobs', refit=False, scoring='accuracy', verbose=3)

In [19]:
cv.best_params_, cv.best_score_

({'hidden__num_units': 50, 'max_epochs': 5, 'update__learning_rate': 0.1},
 0.96340000000000003)

## Regression task

As is known from sklearn, we have separate estimators for classification and regression.

### Data

In [20]:
X, y = make_regression(n_samples=5000)

Note that apart from using `NeuralNetRegressor` instead of `NeuralNetClassifier`, everything is the same. No need to adjust output nonlinearity or objective.

### Define neural network

In [21]:
l0 = InputLayer()
l1 = DenseLayer(l0, num_units=200)
l2 = DenseLayer(l1)

In [22]:
net = NeuralNetRegressor(l2, verbose=1)

In [23]:
net.set_params(update__learning_rate=0.0001)

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=None, num_units=None),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fcf6ac65128>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fcf6ac650f0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

### Train

In [24]:
net.fit(X, y, epochs=20)

# Neural Network with 20401 learnable parameters

## Layer information

|   # | name   |   size |
|----:|:-------|-------:|
|   0 | input  |    100 |
|   1 | dense  |    200 |
|   2 | dense  |      1 |

|   epoch |   train loss |     dur |
|--------:|-------------:|--------:|
|       1 |  35029.88281 | 0.05796 |
|       2 |  33615.28125 | 0.02884 |
|       3 |  27241.64453 | 0.02839 |
|       4 |  11396.54492 | 0.02905 |
|       5 |   1702.15271 | 0.02858 |
|       6 |    424.58511 | 0.02855 |
|       7 |    313.12628 | 0.02822 |
|       8 |    284.63565 | 0.02873 |
|       9 |    266.37469 | 0.02969 |
|      10 |    251.07422 | 0.02893 |
|      11 |    237.28922 | 0.03085 |
|      12 |    224.54858 | 0.02970 |
|      13 |    212.62373 | 0.02841 |
|      14 |    201.36955 | 0.02951 |
|      15 |    190.67888 | 0.02842 |
|      16 |    180.53098 | 0.02922 |
|      17 |    170.88676 | 0.02869 |
|      18 |    161.73845 | 0.02898 |
|      19 |    153.04208 | 0.02910 |
|      20 |    144.7

NeuralNetRegressor(batch_iterator_test=128, batch_iterator_train=128,
          encoder=None,
          layer=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=DenseLayer(W=GlorotUniform(c01b=False, gain=1.0), b=Constant(value=0.0),
      incoming=InputLayer(Xs=None, make_logs=False, name=None, ys=None),
      make_logs=False, name=None, nonlinearity=None, num_units=200),
      make_logs=False, name=None, nonlinearity=Linear(), num_units=1),
          max_epochs=10, objective=MeanSquaredError(),
          on_epoch_finished=(<mink.handlers.PrintTrainProgress object at 0x7fcf6ac65128>,),
          on_training_started=(<mink.handlers.PrintLayerInfo object at 0x7fcf6ac650f0>,),
          session_kwargs=None, update=SGD(learning_rate=0.0001), verbose=1)

## Saving and restoring

### Save previous net

In [25]:
score_before = mean_squared_error(y, net.predict(X))
print(score_before)

139.113633187


In [26]:
with open('mynet.pkl', 'wb') as f:
    pickle.dump(net, f)

### Create a new net with same architecture

In [27]:
with open('mynet.pkl', 'rb') as f:
    new_net = pickle.load(f)

In [28]:
score_after = mean_squared_error(y, new_net.predict(X))
print(score_after)
assert np.isclose(score_before, score_after)

139.113633187
